# Notebook sobre palabras no en diccionario

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import csv
import operator
import jellyfish

def build_dataframe(csv_path):
    with open(csv_path) as f:
        rows = csv.reader(open(csv_path))
    
    header = rows.next()
    header[0] = "order"
    
    data = []
    for row in rows:
        data.append(dict(zip(header, row)))
        
    return pd.DataFrame(data)
    

df = build_dataframe("palabras.csv")

Para buscar palabras cercanas, vamos a utilizar [la métrica de Jaro-Winkler](https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance)

In [2]:
print(help(jellyfish.jaro_winkler))
jellyfish.jaro_winkler(u"q", "atualização".decode("utf-8"))

Help on built-in function jaro_winkler in module jellyfish.cjellyfish:

jaro_winkler(...)
    jaro_winkler(string1, string2, long_tolerance)
    
    Do a Jaro-Winkler string comparison between string1 and string2.

None


0.0

In [3]:
import operator

similarity = jellyfish.jaro_winkler
THRESHOLD = 0.85
palabras = df.Palabra
sugerencias = {}

for i in range(len(palabras)):
    p1 = palabras[i]
    
    palabras_cercanas = []
    
    for j in range(i+1, len(palabras)):
        p2 = palabras[j]
        
        sim = similarity(p1.decode("utf-8"), p2.decode("utf-8"))
        if sim > THRESHOLD:
            palabras_cercanas.append((p2, sim))
    
    palabras_cercanas = sorted(palabras_cercanas, key=operator.itemgetter(1), reverse=True)
    sugerencias[p1] = [p[0] for p in palabras_cercanas]
    if i % 250 == 0:
        print("{} --> recomendadas : {}".format(p1, palabras_cercanas))

q --> recomendadas : []
promo --> recomendadas : [('promos', 0.9666666666666667), ('proximo', 0.9333333333333333), ('primoo', 0.9022222222222223), ('peroo', 0.88), ('prometi', 0.8742857142857143), ('promise', 0.8742857142857143), ('proxima', 0.8533333333333333)]
fran --> recomendadas : [('frank', 0.96), ('francia', 0.9142857142857143), ('frances', 0.9142857142857143), ('francis', 0.9142857142857143), ('franquito', 0.888888888888889), ('francella', 0.888888888888889), ('frak', 0.8833333333333334), ('fria', 0.8666666666666667), ('fana', 0.8500000000000001)]
veia --> recomendadas : [('eia', 0.9166666666666666), ('veci', 0.8666666666666667), ('vaia', 0.8500000000000001)]
muuy --> recomendadas : [('muyy', 0.8666666666666667)]
olivia --> recomendadas : [('olimpia', 0.8944444444444445), ('oliver', 0.8666666666666666)]
edu --> recomendadas : []
jujeños --> recomendadas : [('juje\xc3\xb1o', 0.9714285714285714), ('juje\xc3\xb1a', 0.9095238095238095)]
kk --> recomendadas : []
sobrinita --> recome

In [10]:
def sugerencias_a_string(palabra):
    sug = sugerencias[palabra]
    return u",".join(s.decode("utf-8") for s in sug)

df['sugerencias'] = map(fruta, df.Palabra)
df[["Palabra", "sugerencias"]].to_csv("palabras_y_sugerencias.csv", encoding="utf-8")